In [1]:
"""output(결과) 파일에서 데이터 가공 (교통량, 지체시간)"""

import pandas as pd
import os
"""output 파일 양식 수정"""
output_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_주행행태_Worst)_250819\시뮬레이션 분석 자료_V2.11\중차량,종단경사,교통량\결과파일" # 결과엑셀 파일이 있는 폴더경로까지
output_files = sorted([f for f in os.listdir(output_path) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]))
print("output_files : ", output_files)
mer_list = []
for i in range(len(output_files)):
    work_file = os.path.join(output_path, output_files[i])
    print(f"{work_file} 작업중...")
    df = pd.read_excel(work_file)

    # 신규 그룹 컬럼 생성
    df["NEW"] = df["DATACOLLECTIONMEASUREMENT"] % 1000

    # 기존 DATACOLLECTIONMEASUREMENT 컬럼의 값이 같은 것끼리 평균 계산
    df_grouped = df.groupby("DATACOLLECTIONMEASUREMENT").mean()

    # VEHS(ALL), QUEUEDELAY(ALL) 컬럼의 값만 합 연산, 나머지는 평균
    df_newGrouped = df_grouped.groupby("NEW").agg(lambda x: x.sum() if x.name == "VEHS(ALL)" or  x.name == "QUEUEDELAY(ALL)" else x.mean()).reset_index()

    df_newGrouped["VEHS(ALL)"] = df_newGrouped["VEHS(ALL)"] * 60
    df = df_newGrouped[["NEW", "VEHS(ALL)", "QUEUEDELAY(ALL)"]]
    del df_newGrouped
    display(df)

    # 지상부(진입)
    entry_ground_df = df[(df["NEW"] >= 1) & (df["NEW"] <=10)]
    # 진입부
    start_df = df[(df["NEW"] >= 11) & (df["NEW"] <=40)]
    # 본선부
    mid_df = df[(df["NEW"] >= 41) & (df["NEW"] <=240)]
    # 진출부
    end_df = df[(df["NEW"] >= 241) & (df["NEW"] <=270)]
    # 지상부(진출)
    exit_ground_df = df[(df["NEW"] >= 271) & (df["NEW"] <=280)]
    # 전체
    total_df = df[(df["NEW"] >= 1) & (df["NEW"] <=280)]

    # 구간별로 VEHS(ALL): 평균, QUEUEDELAY(ALL): 합계 처리
    def summarize_section(df_section):
        vehs_mean = df_section["VEHS(ALL)"].mean()
        queuedelay_sum = df_section["QUEUEDELAY(ALL)"].sum()
        return [vehs_mean, queuedelay_sum]

    df1_vals = summarize_section(entry_ground_df)
    df2_vals = summarize_section(start_df)
    df3_vals = summarize_section(mid_df)
    df4_vals = summarize_section(end_df)
    df5_vals = summarize_section(exit_ground_df)
    df6_vals = summarize_section(total_df)

    # 컬럼 정의
    columns = [
        "파일명",
        "VEHS(ALL)_지상(진입)", "VEHS(ALL)_진입", "VEHS(ALL)_본선", "VEHS(ALL)_진출", "VEHS(ALL)_지상(진출)", "VEHS(ALL)_전체",
        "QUEUEDELAY(ALL)_지상(진입)", "QUEUEDELAY(ALL)_진입", "QUEUEDELAY(ALL)_본선", "QUEUEDELAY(ALL)_진출", "QUEUEDELAY(ALL)_지상(진출)", "QUEUEDELAY(ALL)_전체"
    ]
    # 컬럼 순서 정렬
    merged_data = [
        output_files[i],
        df1_vals[0], df2_vals[0], df3_vals[0], df4_vals[0], df5_vals[0], df6_vals[0],  # VEHS(ALL)_*
        df1_vals[1], df2_vals[1], df3_vals[1], df4_vals[1], df5_vals[1], df6_vals[1]  # QUEUEDELAY(ALL)_*
    ]

    merged_df = pd.DataFrame([merged_data], columns=columns)
    mer_list.append(merged_df)


final_df = pd.concat(mer_list, ignore_index=True).round(4)
display(final_df)
excel_path = os.path.join(output_path, "volume_output")
print("excel_path : ", excel_path)

if not os.path.exists(excel_path):
    os.makedirs(excel_path)
excel_file_path = os.path.join(excel_path, "volume_output.xlsx")
final_df.to_excel(excel_file_path, index=False)
print("작업이 완료 되었습니다.")


output_files :  ['1_output.xlsx', '2_output.xlsx', '3_output.xlsx', '4_output.xlsx', '5_output.xlsx', '6_output.xlsx', '7_output.xlsx', '8_output.xlsx', '9_output.xlsx', '10_output.xlsx', '11_output.xlsx', '12_output.xlsx', '13_output.xlsx', '14_output.xlsx', '15_output.xlsx', '16_output.xlsx', '17_output.xlsx', '18_output.xlsx', '19_output.xlsx', '20_output.xlsx']
C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_주행행태_Worst)_250819\시뮬레이션 분석 자료_V2.11\중차량,종단경사,교통량\결과파일\1_output.xlsx 작업중...


,NEW,VEHS(ALL),QUEUEDELAY(ALL)
0,1.0,4997.0,0.000000
1,2.0,4998.0,0.000000
2,3.0,4999.0,0.000000
3,4.0,5000.0,0.000000
4,5.0,4998.0,0.000000
...,...,...,...
275,276.0,4896.0,7.825667
276,277.0,4893.0,7.836000
277,278.0,4888.0,7.840667
278,279.0,4886.0,7.802167


C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_주행행태_Worst)_250819\시뮬레이션 분석 자료_V2.11\중차량,종단경사,교통량\결과파일\2_output.xlsx 작업중...


,NEW,VEHS(ALL),QUEUEDELAY(ALL)
0,1.0,4997.0,0.000000
1,2.0,4998.0,0.000000
2,3.0,4999.0,0.000000
3,4.0,5000.0,0.000000
4,5.0,4998.0,0.000000
...,...,...,...
275,276.0,4896.0,7.825667
276,277.0,4893.0,7.836000
277,278.0,4888.0,7.840667
278,279.0,4886.0,7.802167


C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_주행행태_Worst)_250819\시뮬레이션 분석 자료_V2.11\중차량,종단경사,교통량\결과파일\3_output.xlsx 작업중...


...

,NEW,VEHS(ALL),QUEUEDELAY(ALL)
0,1.0,4997.0,0.000000
1,2.0,4998.0,0.000000
2,3.0,4999.0,0.000000
3,4.0,5000.0,0.000000
4,5.0,4998.0,0.000000
...,...,...,...
275,276.0,4915.0,11.036833
276,277.0,4912.0,11.055833
277,278.0,4906.0,11.037167
278,279.0,4900.0,11.044000


C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_주행행태_Worst)_250819\시뮬레이션 분석 자료_V2.11\중차량,종단경사,교통량\결과파일\20_output.xlsx 작업중...


,NEW,VEHS(ALL),QUEUEDELAY(ALL)
0,1.0,4997.0,0.000000
1,2.0,4998.0,0.000000
2,3.0,4999.0,0.000000
3,4.0,5000.0,0.000000
4,5.0,4998.0,0.000000
...,...,...,...
275,276.0,4915.0,11.036833
276,277.0,4912.0,11.055833
277,278.0,4906.0,11.037167
278,279.0,4900.0,11.044000


,파일명,VEHS(ALL)_지상(진입),VEHS(ALL)_진입,VEHS(ALL)_본선,VEHS(ALL)_진출,VEHS(ALL)_지상(진출),VEHS(ALL)_전체,QUEUEDELAY(ALL)_지상(진입),QUEUEDELAY(ALL)_진입,QUEUEDELAY(ALL)_본선,QUEUEDELAY(ALL)_진출,QUEUEDELAY(ALL)_지상(진출),QUEUEDELAY(ALL)_전체
0,1_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
1,2_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
2,3_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
3,4_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
4,5_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
5,6_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
6,7_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
7,8_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
8,9_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962
9,10_output.xlsx,4997.6,4995.3333,5023.980,4929.6667,4896.4,5005.3071,0.0,0.0,16.4085,285.5007,79.7870,381.6962


excel_path :  C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_주행행태_Worst)_250819\시뮬레이션 분석 자료_V2.11\중차량,종단경사,교통량\결과파일\volume_output
작업이 완료 되었습니다.
